<a href="https://colab.research.google.com/github/caue22/TCC_Ransonware/blob/main/Algoritmos_Colab/Treinando_Modelo_Random_Forenst_Balanceado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline  # Para usar RandomUnderSampler no pipeline
import joblib

import pandas as pd
from sklearn.utils import resample

# Carregar o dataset
file_path = '/content/drive/MyDrive/Dataset Ransonware/ed0e3c43-8495-45d6-9fe1-c91f36b9c82b_shuffled_undersampled.csv'
df = pd.read_csv(file_path)

# Verificar as classes e a quantidade de registros em cada classe
class_column = 'family'  # Substitua pelo nome correto da coluna de classe
class_counts = df[class_column].value_counts()
print("Contagem de registros por classe:\n", class_counts)

# Balancear o dataset por undersampling (número de registros igual ao da classe menor)
min_class_count = class_counts.min()
df_balanced = pd.DataFrame()

# Resample para cada classe
for label in class_counts.index:
    df_class = df[df[class_column] == label]
    df_class_resampled = resample(df_class, replace=False, n_samples=min_class_count, random_state=42)
    df_balanced = pd.concat([df_balanced, df_class_resampled])

# Embaralhar o dataset balanceado
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Salvar o dataset balanceado em CSV
output_file_path = '/content/drive/MyDrive/Dataset Ransonware/ed0e3c43-8495-45d6-9fe1-c91f36b9c82b_balanced.csv'
df_balanced.to_csv(output_file_path, index=False)

print("Dataset balanceado salvo em:", output_file_path)


# Carregar o conjunto de dados
file_path = '/content/drive/MyDrive/Dataset Ransonware/ed0e3c43-8495-45d6-9fe1-c91f36b9c82b_balanced.csv'
df = pd.read_csv(file_path, low_memory=False)

# Separar características e rótulos
X = df.iloc[:, 2:]  # Características (todas as colunas, exceto a primeira e a segunda)
y = df.iloc[:, 1]   # Rótulo (segunda coluna)

# Verificar tipos de dados e garantir que são numéricos
X = X.select_dtypes(include=[float, int])
print(f"Número de amostras em X: {X.shape[0]}")
print(f"Número de amostras em y: {y.shape[0]}")

# Codificar a coluna de rótulo para números
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Salvar o LabelEncoder para uso posterior no Google Drive
label_encoder_path = '/content/drive/MyDrive/Modelos/label_encoder.pkl'
joblib.dump(label_encoder, label_encoder_path)
print(f"Label encoder salvo em: {label_encoder_path}")

# Dividir o conjunto de dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Criar pipeline com normalização e undersampling
pipeline = ImbPipeline(steps=[
    ('scaler', StandardScaler()),  # Normalizar os dados
    ('undersampler', RandomUnderSampler(random_state=42)),  # Balancear o conjunto de treino
    ('classifier', RandomForestClassifier(random_state=42))  # Classificador
])

# Treinar o modelo no conjunto de dados de treino balanceado e normalizado
pipeline.fit(X_train, y_train)

# Avaliar a precisão do modelo no conjunto de teste
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Salvar o modelo treinado no Google Drive
model_path = '/content/drive/MyDrive/Modelos/random_forest_model.pkl'
joblib.dump(pipeline, model_path)
print(f"Modelo salvo em: {model_path}")

# Salvar o conjunto de teste em CSV com índice sequencial e rótulo verdadeiro
X_test_with_labels = X_test.copy()
X_test_with_labels['index'] = range(len(X_test_with_labels))  # Criar índice sequencial
X_test_with_labels['true_label'] = label_encoder.inverse_transform(y_test)  # Adicionar os rótulos verdadeiros

# Salvar em CSV no Google Drive
csv_test_path = '/content/drive/MyDrive/Modelos/conjunto_teste.csv'
X_test_with_labels.to_csv(csv_test_path, index=False)
print(f"Conjunto de teste salvo como: {csv_test_path}")


Número de amostras em X: 175566
Número de amostras em y: 175566
Label encoder salvo em: /content/drive/MyDrive/Modelos/label_encoder.pkl
Accuracy: 96.64%
Modelo salvo em: /content/drive/MyDrive/Modelos/random_forest_model.pkl
Conjunto de teste salvo como: /content/drive/MyDrive/Modelos/conjunto_teste.csv


In [ ]:
# Carregar o dataset
file_path = '/content/drive/MyDrive/Dataset Ransonware/ed0e3c43-8495-45d6-9fe1-c91f36b9c82b_balanced.csv'
df = pd.read_csv(file_path)

# Verificar as classes e a quantidade de registros em cada classe
class_column = 'family'  # Substitua pelo nome correto da coluna de classe
class_counts = df[class_column].value_counts()
print("Contagem de registros por classe:\n", class_counts)

NameError: name 'pd' is not defined

In [ ]:
# Verificar a quantidade de registros de cada classe codificada
import numpy as np

# Para os rótulos originais antes da codificação
print(y.value_counts())

# Para os rótulos codificados
unique, counts = np.unique(y_encoded, return_counts=True)
class_counts = dict(zip(label_encoder.inverse_transform(unique), counts))
print(class_counts)


family
E    58522
L    58522
G    58522
Name: count, dtype: int64
{'E': 58522, 'G': 58522, 'L': 58522}


In [ ]:
quai# Realizar previsões no conjunto de teste
y_test_pred = pipeline.predict(X_test)

# Verificar a precisão do modelo no conjunto de teste
accuracy = accuracy_score(y_test, y_test_pred)
print(f"Accuracy no conjunto de teste: {accuracy * 100:.2f}%")

# Comparar previsões com rótulos verdadeiros
comparison_df = pd.DataFrame({
    'Index': range(len(y_test)),  # Índices das amostras
    'True Label': label_encoder.inverse_transform(y_test),  # Rótulos verdadeiros
    'Predicted Label': label_encoder.inverse_transform(y_test_pred)  # Rótulos previstos
})

# Exibir algumas previsões para verificar se o modelo está acertando
print(comparison_df.head(10))  # Exibir as primeiras 10 previsões

# Opcional: Exibir todos os índices onde as previsões estão incorretas
incorrect_predictions = comparison_df[comparison_df['True Label'] != comparison_df['Predicted Label']]
print(f"Amostras incorretas:\n{incorrect_predictions}")


Accuracy no conjunto de teste: 96.64%
   Index True Label Predicted Label
0      0          E               E
1      1          L               L
2      2          E               E
3      3          L               L
4      4          G               G
5      5          L               G
6      6          E               E
7      7          G               G
8      8          L               L
9      9          L               L
Amostras incorretas:
       Index True Label Predicted Label
5          5          L               G
91        91          G               L
104      104          E               G
106      106          E               G
181      181          L               G
...      ...        ...             ...
35035  35035          G               E
35060  35060          L               G
35066  35066          L               G
35087  35087          L               G
35111  35111          L               G

[1180 rows x 3 columns]


Contagem de registros por classe:
 family
E    1276865
G      85231
L      58522
Name: count, dtype: int64
Dataset balanceado salvo em: /content/drive/MyDrive/Dataset Ransonware/ed0e3c43-8495-45d6-9fe1-c91f36b9c82b_balanced.csv
